# 畳み込み操作をステップバイステップで理解する

フィルタが画像上をどのようにスライドして畳み込むか、1ステップずつ可視化します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.colors import LinearSegmentedColormap

## 1. シンプルな例: 5×5画像に3×3フィルタ

まず小さな例で畳み込みの基本を理解しましょう。

In [ ]:
# 5×5の入力画像（シンプルな例）
input_image = np.array([
    [0,   0,   0,   0,   0],
    [0, 100, 100, 100,   0],
    [0, 100,   0, 100,   0],
    [0, 100, 100, 100,   0],
    [0,   0,   0,   0,   0]
])

# 3×3フィルタ（エッジ検出）
filter_3x3 = np.array([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
])

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# 入力画像
ax = axes[0]
im = ax.imshow(input_image, cmap='gray', vmin=0, vmax=255)
ax.set_title('Input Image (5×5)', fontsize=12, fontweight='bold')
for i in range(5):
    for j in range(5):
        ax.text(j, i, f'{input_image[i,j]}', ha='center', va='center', 
                color='red' if input_image[i,j] > 50 else 'white', fontsize=10)
ax.set_xticks(range(5))
ax.set_yticks(range(5))

# フィルタ
ax = axes[1]
im = ax.imshow(filter_3x3, cmap='RdBu', vmin=-8, vmax=8)
ax.set_title('Filter (3×3)', fontsize=12, fontweight='bold')
for i in range(3):
    for j in range(3):
        ax.text(j, i, f'{filter_3x3[i,j]}', ha='center', va='center', fontsize=12, fontweight='bold')
ax.set_xticks(range(3))
ax.set_yticks(range(3))
plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

## 2. フィルタのスライド操作を可視化

フィルタが画像上を**左から右へ、上から下へ**順番にスライドします。

In [ ]:
def visualize_convolution_step(input_img, filter_kernel, row, col):
    """
    畳み込みの1ステップを可視化
    
    Args:
        input_img: 入力画像
        filter_kernel: フィルタ
        row, col: フィルタの左上位置
    """
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    fh, fw = filter_kernel.shape
    
    # 1. 入力画像とフィルタ位置
    ax = axes[0]
    ax.imshow(input_img, cmap='gray', vmin=0, vmax=255)
    rect = Rectangle((col-0.5, row-0.5), fw, fh, fill=False, 
                      edgecolor='red', linewidth=3)
    ax.add_patch(rect)
    for i in range(input_img.shape[0]):
        for j in range(input_img.shape[1]):
            color = 'yellow' if (row <= i < row+fh and col <= j < col+fw) else 'white'
            ax.text(j, i, f'{input_img[i,j]}', ha='center', va='center', 
                    color=color, fontsize=9, fontweight='bold')
    ax.set_title(f'Step: Filter at ({row},{col})', fontsize=11, fontweight='bold')
    ax.set_xticks(range(input_img.shape[1]))
    ax.set_yticks(range(input_img.shape[0]))
    
    # 2. 切り出した領域
    ax = axes[1]
    region = input_img[row:row+fh, col:col+fw]
    ax.imshow(region, cmap='gray', vmin=0, vmax=255)
    for i in range(fh):
        for j in range(fw):
            ax.text(j, i, f'{region[i,j]}', ha='center', va='center', 
                    color='yellow', fontsize=12, fontweight='bold')
    ax.set_title('Extracted Region', fontsize=11, fontweight='bold')
    ax.set_xticks(range(fw))
    ax.set_yticks(range(fh))
    
    # 3. 要素ごとの掛け算
    ax = axes[2]
    product = region * filter_kernel
    ax.imshow(np.zeros_like(product), cmap='gray', vmin=-1, vmax=1)
    for i in range(fh):
        for j in range(fw):
            ax.text(j, i, f'{region[i,j]}×{filter_kernel[i,j]}\n={product[i,j]:.0f}', 
                    ha='center', va='center', color='cyan', fontsize=9)
    ax.set_title('Element-wise Multiply\n(同じ位置同士を掛ける)', fontsize=11, fontweight='bold')
    ax.set_xticks(range(fw))
    ax.set_yticks(range(fh))
    
    # 4. 合計（出力値）
    ax = axes[3]
    total = np.sum(product)
    ax.text(0.5, 0.6, f'Sum = {total:.0f}', ha='center', va='center', 
            fontsize=20, fontweight='bold', transform=ax.transAxes)
    ax.text(0.5, 0.3, f'Output[{row},{col}] = {total:.0f}', ha='center', va='center', 
            fontsize=14, color='red', transform=ax.transAxes)
    ax.axis('off')
    ax.set_title('Sum All\n(全部足す)', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return total

# 最初のステップ（左上）
print("=" * 60)
print("位置 (0,0): フィルタが左上にある状態")
print("=" * 60)
result_00 = visualize_convolution_step(input_image, filter_3x3, 0, 0)

In [ ]:
# 2番目のステップ（1つ右へ移動）
print("=" * 60)
print("位置 (0,1): フィルタを1ピクセル右へ移動")
print("=" * 60)
result_01 = visualize_convolution_step(input_image, filter_3x3, 0, 1)

In [ ]:
# 3番目のステップ（さらに右へ）
print("=" * 60)
print("位置 (0,2): フィルタをさらに1ピクセル右へ（この行の最後）")
print("=" * 60)
result_02 = visualize_convolution_step(input_image, filter_3x3, 0, 2)

In [ ]:
# 4番目のステップ（次の行へ）
print("=" * 60)
print("位置 (1,0): 次の行の左端へ移動")
print("=" * 60)
result_10 = visualize_convolution_step(input_image, filter_3x3, 1, 0)

## 3. 全スライド位置を一覧表示

In [ ]:
# 全9ステップを一覧表示
fig, axes = plt.subplots(3, 3, figsize=(12, 12))

output_size = input_image.shape[0] - filter_3x3.shape[0] + 1  # 5 - 3 + 1 = 3
output_map = np.zeros((output_size, output_size))

step = 0
for row in range(output_size):
    for col in range(output_size):
        ax = axes[row, col]
        
        # 入力画像を表示
        ax.imshow(input_image, cmap='gray', vmin=0, vmax=255)
        
        # フィルタ位置を赤枠で表示
        rect = Rectangle((col-0.5, row-0.5), 3, 3, fill=False, 
                          edgecolor='red', linewidth=3)
        ax.add_patch(rect)
        
        # 値を表示
        for i in range(5):
            for j in range(5):
                color = 'yellow' if (row <= i < row+3 and col <= j < col+3) else 'gray'
                ax.text(j, i, f'{input_image[i,j]}', ha='center', va='center', 
                        color=color, fontsize=8)
        
        # 畳み込み計算
        region = input_image[row:row+3, col:col+3]
        result = np.sum(region * filter_3x3)
        output_map[row, col] = result
        
        step += 1
        ax.set_title(f'Step {step}: ({row},{col})\nOutput = {result:.0f}', 
                     fontsize=10, fontweight='bold')
        ax.set_xticks([])
        ax.set_yticks([])

plt.suptitle('Filter Sliding: All 9 Positions\n(フィルタが9箇所をスライド)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 出力特徴マップを表示
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# 入力
ax = axes[0]
ax.imshow(input_image, cmap='gray', vmin=0, vmax=255)
ax.set_title('Input (5×5)', fontsize=12, fontweight='bold')
for i in range(5):
    for j in range(5):
        ax.text(j, i, f'{input_image[i,j]}', ha='center', va='center', 
                color='red' if input_image[i,j] > 50 else 'white', fontsize=9)

# フィルタ
ax = axes[1]
ax.imshow(filter_3x3, cmap='RdBu', vmin=-8, vmax=8)
ax.set_title('Filter (3×3)', fontsize=12, fontweight='bold')
for i in range(3):
    for j in range(3):
        ax.text(j, i, f'{filter_3x3[i,j]}', ha='center', va='center', fontsize=11)

# 出力
ax = axes[2]
im = ax.imshow(output_map, cmap='RdBu')
ax.set_title('Output Feature Map (3×3)', fontsize=12, fontweight='bold')
for i in range(3):
    for j in range(3):
        ax.text(j, i, f'{output_map[i,j]:.0f}', ha='center', va='center', 
                fontsize=11, fontweight='bold')
plt.colorbar(im, ax=ax)

# 矢印
axes[0].annotate('', xy=(1.15, 0.5), xytext=(1.02, 0.5),
                 xycoords='axes fraction', textcoords='axes fraction',
                 arrowprops=dict(arrowstyle='->', color='green', lw=2))
axes[1].annotate('', xy=(1.15, 0.5), xytext=(1.02, 0.5),
                 xycoords='axes fraction', textcoords='axes fraction',
                 arrowprops=dict(arrowstyle='->', color='green', lw=2))

plt.suptitle('Convolution Result: 5×5 input + 3×3 filter → 3×3 output', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"出力サイズの計算: (5 - 3) + 1 = 3")
print(f"つまり: (入力サイズ - フィルタサイズ) + 1 = 出力サイズ")

## 4. スライドの順番をアニメーション風に

In [ ]:
# スライドの順番を矢印で表示
fig, ax = plt.subplots(figsize=(8, 8))

# 入力画像
ax.imshow(input_image, cmap='gray', vmin=0, vmax=255)

# グリッド
for i in range(6):
    ax.axhline(y=i-0.5, color='gray', linewidth=0.5, alpha=0.5)
    ax.axvline(x=i-0.5, color='gray', linewidth=0.5, alpha=0.5)

# フィルタのスライド経路を矢印で表示
positions = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
colors = plt.cm.rainbow(np.linspace(0, 1, len(positions)))

for idx, (row, col) in enumerate(positions):
    # フィルタの中心位置
    center_y = row + 1
    center_x = col + 1
    
    # 番号を表示
    ax.text(center_x, center_y, f'{idx+1}', ha='center', va='center', 
            fontsize=16, fontweight='bold', color='red',
            bbox=dict(boxstyle='circle', facecolor='yellow', edgecolor='red', linewidth=2))
    
    # 矢印（次の位置へ）
    if idx < len(positions) - 1:
        next_row, next_col = positions[idx + 1]
        next_center_y = next_row + 1
        next_center_x = next_col + 1
        
        # 同じ行内なら右へ、次の行なら左下へ
        ax.annotate('', xy=(next_center_x - 0.3, next_center_y), 
                    xytext=(center_x + 0.3, center_y),
                    arrowprops=dict(arrowstyle='->', color=colors[idx], lw=2))

ax.set_title('Filter Sliding Order\n(フィルタのスライド順序: 左→右、上→下)', 
             fontsize=14, fontweight='bold')
ax.set_xlim(-0.5, 4.5)
ax.set_ylim(4.5, -0.5)
ax.set_xticks(range(5))
ax.set_yticks(range(5))

plt.tight_layout()
plt.show()

print("スライド順序:")
print("1→2→3 (1行目を左から右へ)")
print("   ↓")
print("4→5→6 (2行目を左から右へ)")
print("   ↓")
print("7→8→9 (3行目を左から右へ)")

## 5. Stride（ストライド）の違い

フィルタを何ピクセルずつ移動するかを「ストライド」といいます。

In [ ]:
# 7×7の入力画像
input_7x7 = np.zeros((7, 7))
input_7x7[1:6, 1:6] = 100
input_7x7[2:5, 2:5] = 200

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

strides = [1, 2, 3]
titles = ['Stride=1\n(1ピクセルずつ)', 'Stride=2\n(2ピクセルずつ)', 'Stride=3\n(3ピクセルずつ)']

for idx, (stride, title) in enumerate(zip(strides, titles)):
    ax = axes[idx]
    ax.imshow(input_7x7, cmap='gray', vmin=0, vmax=255)
    
    # フィルタ位置を表示
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'brown']
    pos_idx = 0
    output_size = (7 - 3) // stride + 1
    
    for row in range(0, 7 - 2, stride):
        for col in range(0, 7 - 2, stride):
            if row + 3 <= 7 and col + 3 <= 7:
                rect = Rectangle((col-0.5, row-0.5), 3, 3, fill=False, 
                                  edgecolor=colors[pos_idx % len(colors)], 
                                  linewidth=2, linestyle='--')
                ax.add_patch(rect)
                ax.text(col+1, row+1, f'{pos_idx+1}', ha='center', va='center', 
                        fontsize=12, fontweight='bold', color=colors[pos_idx % len(colors)])
                pos_idx += 1
    
    ax.set_title(f'{title}\nOutput: {output_size}×{output_size}', fontsize=11, fontweight='bold')
    ax.set_xticks(range(7))
    ax.set_yticks(range(7))

plt.suptitle('Stride Comparison (7×7 input, 3×3 filter)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("出力サイズの計算式: (入力サイズ - フィルタサイズ) / ストライド + 1")
print(f"Stride=1: (7-3)/1 + 1 = 5")
print(f"Stride=2: (7-3)/2 + 1 = 3")
print(f"Stride=3: (7-3)/3 + 1 = 2 (小数点切り捨て)")

## 6. 「8」の画像での畳み込みプロセス

In [ ]:
# 「8」の画像を作成（12×12に縮小）
def create_digit_8_small():
    img = np.zeros((12, 12))
    
    # 上の丸
    cy1, cx = 3, 6
    for i in range(12):
        for j in range(12):
            dist = np.sqrt((i - cy1)**2 + (j - cx)**2)
            if 1.5 < dist < 2.8:
                img[i, j] = 200
    
    # 下の丸
    cy2 = 8
    for i in range(12):
        for j in range(12):
            dist = np.sqrt((i - cy2)**2 + (j - cx)**2)
            if 1.8 < dist < 3:
                img[i, j] = 200
    
    return img

digit_8_small = create_digit_8_small()

# 縦エッジフィルタ
vertical_filter = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
]) / 4

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(digit_8_small, cmap='gray', vmin=0, vmax=255)
plt.title('"8" Image (12×12)', fontsize=12, fontweight='bold')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(vertical_filter, cmap='RdBu', vmin=-1, vmax=1)
plt.title('Vertical Edge Filter (3×3)', fontsize=12, fontweight='bold')
for i in range(3):
    for j in range(3):
        plt.text(j, i, f'{vertical_filter[i,j]:.1f}', ha='center', va='center', fontsize=10)
plt.colorbar()

plt.tight_layout()
plt.show()

In [ ]:
# 畳み込みの特定位置を可視化
from scipy.ndimage import convolve

# 畳み込み結果
conv_result = convolve(digit_8_small, vertical_filter)

# 4つの位置でフィルタの動作を表示
positions_to_show = [(2, 3), (2, 8), (5, 3), (7, 8)]  # 左上、右上、左中、右下

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, (row, col) in enumerate(positions_to_show):
    # 上段: フィルタ位置
    ax = axes[0, idx]
    ax.imshow(digit_8_small, cmap='gray', vmin=0, vmax=255)
    rect = Rectangle((col-0.5, row-0.5), 3, 3, fill=False, 
                      edgecolor='red', linewidth=2)
    ax.add_patch(rect)
    ax.set_title(f'Position ({row},{col})', fontsize=10, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # 下段: 切り出した領域と計算
    ax = axes[1, idx]
    region = digit_8_small[row:row+3, col:col+3]
    product = region * vertical_filter
    result = np.sum(product)
    
    ax.imshow(region, cmap='gray', vmin=0, vmax=255)
    for i in range(3):
        for j in range(3):
            ax.text(j, i, f'{region[i,j]:.0f}', ha='center', va='center', 
                    color='yellow', fontsize=8)
    ax.set_title(f'Region × Filter\n= {result:.1f}', fontsize=10, fontweight='bold',
                 color='red' if abs(result) > 50 else 'black')
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle('Convolution at Different Positions on "8"', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("エッジがある場所（8の輪郭部分）では出力値が大きくなる")
print("エッジがない場所（背景や8の内側）では出力値が小さい")

In [ ]:
# 全体の畳み込み結果を表示
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# 入力
ax = axes[0]
ax.imshow(digit_8_small, cmap='gray', vmin=0, vmax=255)
ax.set_title('Input: "8"\n(12×12)', fontsize=12, fontweight='bold')

# フィルタ
ax = axes[1]
ax.imshow(vertical_filter, cmap='RdBu', vmin=-1, vmax=1)
ax.set_title('Filter: Vertical Edge\n(3×3)', fontsize=12, fontweight='bold')
for i in range(3):
    for j in range(3):
        ax.text(j, i, f'{vertical_filter[i,j]:.1f}', ha='center', va='center', fontsize=10)

# 出力
ax = axes[2]
im = ax.imshow(conv_result, cmap='RdBu')
ax.set_title('Output: Feature Map\n(10×10)', fontsize=12, fontweight='bold')
plt.colorbar(im, ax=ax)

# 矢印
for i in range(2):
    axes[i].annotate('', xy=(1.1, 0.5), xytext=(1.02, 0.5),
                     xycoords='axes fraction', textcoords='axes fraction',
                     arrowprops=dict(arrowstyle='->', color='green', lw=2))

plt.suptitle('Complete Convolution: Filter slides across entire image', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("青い部分: 左側のエッジを検出（明→暗の境界）")
print("赤い部分: 右側のエッジを検出（暗→明の境界）")
print("白い部分: エッジなし（平坦な領域）")

## まとめ: 畳み込みの流れ

```
1. フィルタを画像の左上に配置
        ↓
2. フィルタと画像の重なる部分で「同じ位置同士を掛けて全部足す」
        ↓
3. 結果を出力マップの対応位置に記録
        ↓
4. フィルタを右に1ピクセル移動（stride=1の場合）
        ↓
5. 行の端まで来たら、次の行の左端へ
        ↓
6. 画像全体をカバーするまで繰り返し
```

**ポイント**:
- フィルタは「左→右、上→下」の順でスライド
- 出力サイズ = (入力サイズ - フィルタサイズ) / ストライド + 1
- 各位置で「要素ごとの掛け算 → 合計」を計算